# Analisando os resultados

In [1]:
EXPERIMENTS_PATH = '../TCC/experiments'
EXPERIMENT_NUMBER = 1
import pandas as pd
import numpy as np
import os

### Experimento 1

#### Primeiros passos

In [2]:
YEAR = 2023
df = pd.read_csv(f'{EXPERIMENTS_PATH}/{EXPERIMENT_NUMBER}/{YEAR}/enem_2023.csv')
df.head()

,id,year,area,sp_area,day,body,alternatives,answer,GEMINI_2023_answers,GPT-3.5-turbo-answers,GPT3.5_2023_answers,Llama2_2023_answers
0,01I,2023,LC - Linguagens e Códigos,Inglês,D1,"QUESTÃO 01 \nNo man is an island,\nEntire of ...",\nA medo da morte.\nB ideia de conexão.\nC con...,B,B,B,B,B
1,02I,2023,LC - Linguagens e Códigos,Inglês,D1,QUESTÃO 02 \nThings We Carry on the Sea\nWe c...,\nA risco de choques culturais.\nB impacto do ...,D,D,D,D,D
2,03I,2023,LC - Linguagens e Códigos,Inglês,D1,QUESTÃO 03 \nDescrição do cartaz: Foto de par...,\nA os lixões precisam de ampliação. \nB o des...,B,B,B,B,B
3,04I,2023,LC - Linguagens e Códigos,Inglês,D1,QUESTÃO 04 \nDescrição do cartum: Em um escri...,\nA criticar um padrão de vestimenta.\nB desta...,B,B,B,C,C
4,05I,2023,LC - Linguagens e Códigos,Inglês,D1,QUESTÃO 05 \nSpanglish (fragmento)\nTato Lavi...,\nA convergência linguístico-cultural.\nB cara...,A,A,A,A,A


In [3]:
df['GPT_result'] = df['GPT3.5_2023_answers'] == df['answer']
df['GPT_result'].value_counts()

GPT_result
True     110
False     75
Name: count, dtype: int64

In [4]:
df['GPT_UI_result'] = df['GPT-3.5-turbo-answers'] == df['answer']
df['GPT_UI_result'].value_counts()

GPT_UI_result
True     128
False     57
Name: count, dtype: int64

In [5]:
df['Llama2_result'] = df['Llama2_2023_answers'] == df['answer']
df['Llama2_result'].value_counts()

Llama2_result
False    95
True     90
Name: count, dtype: int64

In [6]:
df['GEMINI_result'] = df['GEMINI_2023_answers'] == df['answer']
df['GEMINI_result'].value_counts()

GEMINI_result
True     132
False     53
Name: count, dtype: int64

In [7]:
similar_answers = df[(df['Llama2_2023_answers'] == df['GPT-3.5-turbo-answers']) & (df['GPT-3.5-turbo-answers'] == df['GEMINI_2023_answers'])]
len(similar_answers)

78

In [8]:
df['answer'].value_counts()

answer
A          46
C          38
D          37
B          34
E          29
Anulado     1
Name: count, dtype: int64

In [9]:
df.Llama2_2023_answers.value_counts()

Llama2_2023_answers
C    66
B    46
D    34
E    27
A    12
Name: count, dtype: int64

In [10]:
display(df.query('Llama2_result == True ')['Llama2_2023_answers'].value_counts())

Llama2_2023_answers
C    27
B    20
D    19
A    12
E    12
Name: count, dtype: int64

#### Análise 2018 - 2023

In [11]:
df_2018 = pd.read_csv(f'{EXPERIMENTS_PATH}/{EXPERIMENT_NUMBER}/2018/enem_2018.csv')
display(df_2018.head())
df_2019 = pd.read_csv(f'{EXPERIMENTS_PATH}/{EXPERIMENT_NUMBER}/2019/enem_2019.csv')
# display(df_2019.head())
df_2020 = pd.read_csv(f'{EXPERIMENTS_PATH}/{EXPERIMENT_NUMBER}/2020/enem_2020.csv')
# display(df_2020.head())   
df_2022 = pd.read_csv(f'{EXPERIMENTS_PATH}/{EXPERIMENT_NUMBER}/2022/enem_2022.csv')
# display(df_2022.head())
df_2023 = pd.read_csv(f'{EXPERIMENTS_PATH}/{EXPERIMENT_NUMBER}/2023/enem_2023.csv')
# display(df_2023.head())   

,id,year,area,sp_area,day,body,alternatives,answer,GEMINI_2018_answers,GPT3.5_2018_answers,Llama2_2018_answers
0,01I,2018,LC - Linguagens e Códigos,Inglês,D1,QUESTÃO 01 \nLava Mae: Creating Showers on Wh...,\nA empregar moradores de rua em lava a jatos ...,B,B,B,B
1,02I,2018,LC - Linguagens e Códigos,Inglês,D1,QUESTÃO 02 \nDescrição do cartum:\nO cartum a...,\nA se aposente prematuramente.\nB amadureça p...,B,B,B,B
2,03I,2018,LC - Linguagens e Códigos,Inglês,D1,"QUESTÃO 03 \nDon't write in English, they sai...",\nA usa a língua inglesa com efeito humorístic...,D,D,D,D
3,04I,2018,LC - Linguagens e Códigos,Inglês,D1,QUESTÃO 04 \nTEXTO I\n A Free World-class Edu...,\nA elogiar o trabalho proposto para a educaçã...,E,E,E,E
4,05I,2018,LC - Linguagens e Códigos,Inglês,D1,QUESTÃO 05 \n1984 (excerpt)\nGeorge Orwell\n‘...,\nA boicote a ideais libertários.\nB veto ao c...,C,C,C,C


In [12]:
def check_models_answers(df, year):
    print(df[f'GEMINI_{year}_answers'].value_counts())
    print('Soma das respostas do GEMINI:', df[f'GEMINI_{year}_answers'].value_counts().sum())
    print(df[f'GPT3.5_{year}_answers'].value_counts())
    print('Soma das respostas do GPT3.5:', df[f'GPT3.5_{year}_answers'].value_counts().sum()) 
    print(df[f'Llama2_{year}_answers'].value_counts())
    print('Soma das respostas do Llama2:', df[f'Llama2_{year}_answers'].value_counts().sum())

In [136]:
def check_models_results(df, year):
    df['GPT_result'] = np.where(df[f'GPT3.5_{year}_answers'] == 'No answer', 'No answer', df[f'GPT3.5_{year}_answers'] == df['answer'])
    print(df['GPT_result'].value_counts())
    df['Llama2_result'] = np.where(df[f'Llama2_{year}_answers'] == 'No answer', 'No answer', df[f'Llama2_{year}_answers'] == df['answer'])
    print(df['Llama2_result'].value_counts())
    df['GEMINI_result'] = np.where(df[f'GEMINI_{year}_answers'] == 'No answer', 'No answer', df[f'GEMINI_{year}_answers'] == df['answer'])
    print(df['GEMINI_result'].value_counts())


In [191]:
def save_models_results(df, year):
    
    result_counts_gemini = df['GEMINI_result'].value_counts()
    result_counts_gemini = pd.DataFrame(result_counts_gemini).transpose()
    
    if ('No answer' in result_counts_gemini.columns):
        result_counts_gemini = result_counts_gemini.rename(columns={'True': 'Right', 'False': 'Wrong'})
    else: 
        result_counts_gemini = result_counts_gemini.rename(columns={True: 'Right', False: 'Wrong'})
    result_counts_gemini['Year'] = year
    result_counts_gemini['Model'] = 'GEMINI'

    result_counts_gpt = df['GPT_result'].value_counts()
    result_counts_gpt = pd.DataFrame(result_counts_gpt).transpose()
    
    if('No answer' in result_counts_gpt.columns):
        result_counts_gpt = result_counts_gpt.rename(columns={'True': 'Right', 'False': 'Wrong'})
    else:
        result_counts_gpt = result_counts_gpt.rename(columns={True: 'Right', False: 'Wrong'})
    result_counts_gpt['Year'] = year
    result_counts_gpt['Model'] = 'GPT3.5'
    
    result_counts_llama = df['Llama2_result'].value_counts()
    result_counts_llama = pd.DataFrame(result_counts_llama).transpose()
    
    if('No answer' in result_counts_llama.columns):
        result_counts_llama = result_counts_llama.rename(columns={'True': 'Right', 'False': 'Wrong'})
    else:
        result_counts_llama = result_counts_llama.rename(columns={True: 'Right', False: 'Wrong'})
    result_counts_llama['Year'] = year
    result_counts_llama['Model'] = 'Llama2'
    
    if('True' in result_counts_llama.columns) or ('False' in result_counts_llama.columns):
        result_counts_llama = result_counts_llama.rename(columns={'True': 'Right', 'False': 'Wrong'})
    if('True' in result_counts_gpt.columns) or ('False' in result_counts_gpt.columns):
        result_counts_gpt = result_counts_gpt.rename(columns={'True': 'Right', 'False': 'Wrong'})
    if('True' in result_counts_gemini.columns) or ('False' in result_counts_gemini.columns):
        result_counts_gemini = result_counts_gemini.rename(columns={'True': 'Right', 'False': 'Wrong'})

    result_counts_total_df = pd.concat([result_counts_gemini, result_counts_gpt, result_counts_llama], ignore_index=True).fillna(0)

    display(result_counts_total_df.head())
    return result_counts_total_df


##### Enem 2018

In [14]:
check_models_answers(df_2018, df_2018['year'].unique()[0])
check_models_results(df_2018, df_2018['year'].unique()[0])

GEMINI_2018_answers
B            43
E            38
A            36
C            35
D            30
No answer     3
Name: count, dtype: int64
Soma das respostas do GEMINI: 185
GPT3.5_2018_answers
E    46
B    42
D    41
C    36
A    20
Name: count, dtype: int64
Soma das respostas do GPT3.5: 185
Llama2_2018_answers
C    69
B    53
E    31
D    25
A     7
Name: count, dtype: int64
Soma das respostas do Llama2: 185
GEMINI_2018_answers
B            43
E            38
A            36
C            35
D            30
No answer     3
Name: count, dtype: int64
GPT_result
True     96
False    89
Name: count, dtype: int64
Llama2_result
False    100
True      85
Name: count, dtype: int64
GEMINI_result
True         120
False         62
No answer      3
Name: count, dtype: int64


In [156]:
enem_2018_results = save_models_results(df_2018, df_2018['year'].unique()[0])

,Year,Model,Right,Wrong,No answer
0,2018,GEMINI,120,62,3.0
1,2018,GPT3.5,96,89,0.0
2,2018,Llama2,85,100,0.0


##### Enem 2019

In [66]:
check_models_answers(df_2019, df_2019['year'].unique()[0])
check_models_results(df_2019, df_2019['year'].unique()[0])

GEMINI_2019_answers
A            45
C            41
B            39
D            35
E            24
No answer     1
Name: count, dtype: int64
Soma das respostas do GEMINI: 185
GPT3.5_2019_answers
D    50
C    39
B    34
E    34
A    28
Name: count, dtype: int64
Soma das respostas do GPT3.5: 185
Llama2_2019_answers
C    67
B    46
E    28
D    26
A    11
      3
l     2
(     1
a     1
Name: count, dtype: int64
Soma das respostas do Llama2: 185
GPT_result
True     107
False     78
Name: count, dtype: int64
Llama2_result
False    106
True      79
Name: count, dtype: int64
GEMINI_result
True         125
False         59
No answer      1
Name: count, dtype: int64


In [157]:
enem_2019_results = save_models_results(df_2019, df_2019['year'].unique()[0])

,Year,Model,Right,Wrong,No answer
0,2019,GEMINI,125,59,1.0
1,2019,GPT3.5,107,78,0.0
2,2019,Llama2,79,106,0.0


##### Enem 2020

In [69]:
check_models_answers(df_2020, df_2020['year'].unique()[0])
check_models_results(df_2020, df_2020['year'].unique()[0])

GEMINI_2020_answers
C            42
A            41
D            41
B            35
E            25
No answer     1
Name: count, dtype: int64
Soma das respostas do GEMINI: 185
GPT3.5_2020_answers
A    50
D    48
C    38
E    26
B    23
Name: count, dtype: int64
Soma das respostas do GPT3.5: 185
Llama2_2020_answers
C    82
D    34
B    28
E    26
A     9
      3
i     1
n     1
p     1
Name: count, dtype: int64
Soma das respostas do Llama2: 185
GPT_result
True     93
False    92
Name: count, dtype: int64
Llama2_result
True     96
False    89
Name: count, dtype: int64
GEMINI_result
True         128
False         56
No answer      1
Name: count, dtype: int64


In [158]:
enem_2020_results = save_models_results(df_2020, df_2020['year'].unique()[0])

,Year,Model,Right,Wrong,No answer
0,2020,GEMINI,128,56,1.0
1,2020,GPT3.5,93,92,0.0
2,2020,Llama2,96,89,0.0


##### Enem 2022

In [76]:
check_models_answers(df_2022, df_2022['year'].unique()[0])
check_models_results(df_2022, df_2022['year'].unique()[0])

GEMINI_2022_answers
A            39
C            34
B            26
D            22
E            13
No answer     3
Name: count, dtype: int64
Soma das respostas do GEMINI: 137
GPT3.5_2022_answers
D    49
C    44
E    40
B    27
A    25
Name: count, dtype: int64
Soma das respostas do GPT3.5: 185
Llama2_2022_answers
C    72
B    38
E    35
D    25
A    12
      1
.     1
i     1
Name: count, dtype: int64
Soma das respostas do Llama2: 185
GPT_result
True     114
False     71
Name: count, dtype: int64
Llama2_result
False    93
True     92
Name: count, dtype: int64
GEMINI_result
False        103
True          79
No answer      3
Name: count, dtype: int64


In [187]:
enem_2022_results = save_models_results(df_2022, df_2022['year'].unique()[0])

,Wrong,Right,No answer,Year,Model
0,103,79,3.0,2022,GEMINI
1,71,114,0.0,2022,GPT3.5
2,93,92,0.0,2022,Llama2


##### Enem 2023

In [ ]:
check_models_answers(df_2023, df_2023['year'].unique()[0])
check_models_results(df_2023, df_2023['year'].unique()[0])

In [192]:
enem_2023_results = save_models_results(df_2023, df_2023['year'].unique()[0])

,Right,Wrong,Year,Model
0,132,53,2023,GEMINI
1,110,75,2023,GPT3.5
2,90,95,2023,Llama2


##### Overall

In [194]:
overall_results = pd.concat([enem_2018_results, enem_2019_results, enem_2020_results, enem_2022_results, enem_2023_results], ignore_index=True).fillna(0)
display(overall_results.head(15))

,Year,Model,Right,Wrong,No answer
0,2018,GEMINI,120,62,3.0
1,2018,GPT3.5,96,89,0.0
2,2018,Llama2,85,100,0.0
3,2019,GEMINI,125,59,1.0
4,2019,GPT3.5,107,78,0.0
5,2019,Llama2,79,106,0.0
6,2020,GEMINI,128,56,1.0
7,2020,GPT3.5,93,92,0.0
8,2020,Llama2,96,89,0.0
9,2022,GEMINI,79,103,3.0


In [205]:
overall_results['Perc_Right'] = round(overall_results['Right'] / (overall_results['Right'] + overall_results['Wrong'] + overall_results['No answer']) * 100, 2)
overall_results['Perc_Right'] = overall_results['Perc_Right'].apply(lambda x: f'{x}%')
display(overall_results.head(15))

,Year,Model,Right,Wrong,No answer,Perc_Right
0,2018,GEMINI,120,62,3.0,64.86%
1,2018,GPT3.5,96,89,0.0,51.89%
2,2018,Llama2,85,100,0.0,45.95%
3,2019,GEMINI,125,59,1.0,67.57%
4,2019,GPT3.5,107,78,0.0,57.84%
5,2019,Llama2,79,106,0.0,42.7%
6,2020,GEMINI,128,56,1.0,69.19%
7,2020,GPT3.5,93,92,0.0,50.27%
8,2020,Llama2,96,89,0.0,51.89%
9,2022,GEMINI,79,103,3.0,42.7%


### Experimento 2

In [31]:
df = pd.read_csv(f'{EXPERIMENTS_PATH}/{2}/{YEAR}/enem_2023.csv')
df.head()

,id,year,area,sp_area,day,body,alternatives,answer,GPT3.5_2023_answers
0,01I,2023,LC - Linguagens e Códigos,Inglês,D1,"QUESTÃO 01 \nNo man is an island,\nEntire of ...",\nA medo da morte.\nB ideia de conexão.\nC con...,B,B
1,02I,2023,LC - Linguagens e Códigos,Inglês,D1,QUESTÃO 02 \nThings We Carry on the Sea\nWe c...,\nA risco de choques culturais.\nB impacto do ...,D,D
2,03I,2023,LC - Linguagens e Códigos,Inglês,D1,QUESTÃO 03 \nDescrição do cartaz: Foto de par...,\nA os lixões precisam de ampliação. \nB o des...,B,B
3,04I,2023,LC - Linguagens e Códigos,Inglês,D1,QUESTÃO 04 \nDescrição do cartum: Em um escri...,\nA criticar um padrão de vestimenta.\nB desta...,B,D
4,05I,2023,LC - Linguagens e Códigos,Inglês,D1,QUESTÃO 05 \nSpanglish (fragmento)\nTato Lavi...,\nA convergência linguístico-cultural.\nB cara...,A,A


In [32]:
df['GPT_result'] = df['GPT3.5_2023_answers'] == df['answer']
df['GPT_result'].value_counts()

GPT_result
True     113
False     72
Name: count, dtype: int64